In [1]:
import os
from matplotlib import pyplot as plt

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["NCCL_BLOCKING_WAIT"] = '0'

#os.environ['NCCL_SOCKET_IFNAME'] = 'lo'

In [2]:
import torch
from torch import tensor
from DiSuQ.Torch.optimization import lossTransition
from DiSuQ import utils
torch.cuda.device_count()

2

In [3]:
from DiSuQ.Torch.models import transmon,fluxonium,zeroPi
from datetime import timedelta
import torch.distributed as dist
from torch.distributed import TCPStore,FileStore
from DiSuQ.Torch.components import DISTRIBUTION_BACKEND

In [4]:
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.nn.parallel import DataParallel as DP

In [13]:
torch.distributed.is_nccl_available(),torch.distributed.is_ucc_available()

(True, False)

In [6]:
torch.set_num_threads(36)
cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cpu = torch.device('cpu')
torch.set_default_device(cuda0)

In [7]:
file_path = '/tmp/filestore'
if os.path.exists(file_path):
    os.remove(file_path)
file_store = FileStore(file_path, world_size)

In [8]:
file_store.add('store_based_barrier_key:1',0)

0

In [9]:
#FileStore("/tmp/filestemp", world_size)

In [10]:
#server_store = dist.TCPStore("localhost", 1234, world_size, True, timedelta(seconds=1),False)

In [17]:
from DiSuQ.Torch.circuit import Charge,Kerman
#basis = [15,15,15]
#flux_profile = Tensor([[0.],[.15],[.30],[.5]])
#circuit = zeroPi(Charge,basis,sparse=False)
basis = [1500]
flux_profile = [dict()]
circuit = transmon(Charge,basis,sparse=False)
flux_profile = tensor([[0.],[.15],[.30],[.5]],device=None)
circuit = fluxonium(Charge,basis,sparse=False)
control_iDs = dict()

In [11]:
hash_store = dist.HashStore()

In [12]:
hash_store.add('store_based_barrier_key:1',0)

0

In [ ]:
torch.cuda.set_device(cuda0)

In [14]:
dist.init_process_group(backend='nccl',store = file_store, world_size= world_size, rank=0, timeout=timedelta(seconds=50))

In [15]:
#dist.init_process_group(backend='nccl')

In [ ]:
module = DDP(circuit,device_ids=[cuda0],output_device=cuda0)

In [ ]:
from DiSuQ.Torch.optimization import GradientDescent

In [ ]:
loss = lossTransition(tensor(5.),tensor(4.5))
optim = GradientDescent(circuit,module,flux_profile,loss)
optim.optimizer = optim.initAlgo(lr=1e-2)
print(circuit.circuitComponents())
dLogs,dParams,dCircuit = optim.optimization(100)

In [ ]:
dLogs

In [ ]:
plt.hist(dLogs['time'])
plt.show()

In [ ]:
dist.destroy_process_group()